<a href="https://colab.research.google.com/github/eriksali/DNN_2023_DL/blob/main/pgd_attacks_epsilon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
import numpy as np
from tqdm import tqdm
from scipy.stats import norm

import math


# Download ResNet18 and set the pretrained parameter to True
resnet18 = models.resnet18(pretrained=True)


# Set up the device (GPU or CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the CIFAR-10 dataset
transform = transforms.Compose([transforms.RandomCrop(32, padding=4),
                                transforms.RandomHorizontalFlip(),
                                transforms.ToTensor(),
                                transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])])
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=128, shuffle=False, num_workers=2)

# Define the ResNet18 model and optimizer
##net = ResNet18()
net = resnet18
net = net.to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)

# Train the ResNet18 model
for epoch in range(1):
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
    train_acc = 100. * correct / total

    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)
            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    test_acc = 100. * correct / total

    print('Epoch [%d/%d] Train Loss: %.3f Train Acc: %.3f Test Loss: %.3f Test Acc: %.3f' % (
        epoch + 1, 200, train_loss, train_acc, test_loss, test_acc))

###############################################################################


###############################################################################

###############################################################################


def certify_sample(model, x, y_pred, sigma, num_samples, beta):
    """
    Computes the certified radius for a single input using randomized smoothing.

    Arguments:
        model: A PyTorch model to certify.
        x: The input tensor to certify.
        y_pred: The model's predicted output for the input x.
        sigma: The standard deviation of the Gaussian noise to add to the input during smoothing.
        num_samples: The number of samples to use for smoothing.
        beta: The confidence level for the certification, which determines the radius of the certified set.

    Returns:
        The certified radius for the input x.
    """

    # Set the model to evaluation mode
    model.eval()

    # Compute the perturbation standard deviation for the given sigma and beta
    delta = sigma * math.sqrt(num_samples - 1) * norm.ppf(1 - beta / 2)

    # Compute the smoothed predictions for the input x
    y_smooth = torch.zeros((num_samples,) + y_pred.shape, dtype=y_pred.dtype, device=y_pred.device)
    for i in range(num_samples):
        noise = torch.randn_like(x) * sigma
        y_smooth[i] = model(x + noise)
    y_smooth = y_smooth.softmax(dim=-1)

    # Compute the predicted class probabilities for the smoothed predictions
    y_bar = y_smooth.mean(dim=0)

    # Compute the distance between the original predictions and the smoothed predictions
    d = (y_pred.softmax(dim=-1) - y_bar).norm(p=2)

    # Compute the certified radius
    r = delta + d.item()

    return r


def certify(model, dataset, sigma, num_samples, beta):
    """
    Computes certified radii for a PyTorch model using randomized smoothing.

    Arguments:
        model: A PyTorch model to certify.
        dataset: A PyTorch dataset containing the data to certify.
        sigma: The standard deviation of the Gaussian noise to add to the input during smoothing.
        num_samples: The number of samples to use for smoothing.
        beta: The confidence level for the certification, which determines the radius of the certified set.

    Returns:
        A numpy array of certified radii, one for each input in the dataset.
    """

    # Set the model to evaluation mode
    model.eval()

    # Determine the size of the dataset
    n = len(dataset)

    # Compute the certified radii for each input in the dataset
    certified_radii = np.zeros(n)
    for i in range(n):
        x, y = dataset[i]
        x = x.unsqueeze(0).to(device)
        y_pred = model(x)
        r = certify_sample(model, x, y_pred, sigma, num_samples, beta)
        certified_radii[i] = r

    return certified_radii


# Generate the certified radii for the ResNet18 model on CIFAR-10

from torchvision.datasets import CIFAR10
from torchvision.transforms import ToTensor

# Load the CIFAR-10 test dataset
testset = CIFAR10(root='./data', train=False, download=True, transform=ToTensor())

# Create a data loader for the test dataset
testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=False)

# Define the standard deviation, number of samples, and confidence level for certification
sigma = 0.25
num_samples = 100
beta = 0.001

# Call the certify function with the test dataset and the certification parameters
certified_radii = certify(net, testset, sigma, num_samples, beta)

###############################################################################


def pgd_attack(model, images, labels, eps, alpha, iters, device):
    """
    PGD attack function for adversarial examples generation

    Args:
    - model: the neural network model to attack
    - images: the clean images to be attacked
    - labels: the true labels of the images
    - eps: the maximum L-infinity norm of the perturbations
    - alpha: the step size of each iteration of the attack
    - iters: the number of iterations of the attack
    - device: the device to run the attack on (e.g., 'cuda' for GPU)

    Returns:
    - adv_images: the generated adversarial examples
    """
    # Set model to evaluation mode
    model.eval()

    # Generate random perturbations
    delta = torch.zeros_like(images).uniform_(-eps, eps)
    delta = torch.clamp(images + delta, min=0, max=1) - images
    delta.requires_grad = True

    for i in range(iters):
        # Forward pass
        outputs = model(images + delta)

        # Calculate loss
        loss = nn.CrossEntropyLoss()(outputs, labels)

        # Backward pass
        model.zero_grad()
        loss.backward()

        # Generate adversarial perturbation
        with torch.no_grad():
            delta += alpha * delta.grad.sign()
            delta = torch.clamp(delta, min=-eps, max=eps)
            delta = torch.clamp(images + delta, min=0, max=1) - images
            delta.requires_grad = True

    # Return the adversarial examples
    adv_images = torch.clamp(images + delta, min=0, max=1)

    return adv_images


epsilons = [0.0005, 0.001, 0.002, 0.005, 0.01, 0.02, 0.04, 0.08, 0.16, 0.32]
for epsilon in epsilons:
    print(f"PGD Attack with epsilon={epsilon}")
    correct = 0
    total = 0
    net.eval()
    for batch_idx, (inputs, targets) in enumerate(testloader):
        inputs, targets = inputs.to(device), targets.to(device)
        adv_inputs = pgd_attack(net, inputs, targets, epsilon, 0.01, 2, 1) # Use PGD to generate adversarial examples
        outputs = net(adv_inputs)
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
    acc = 100. * correct / total
    print(f"Accuracy under PGD attack with epsilon {epsilon} : {acc:.2f} %")




In [6]:
##certified_radii = certify(net, testset, sigma, num_samples, beta)
print(certified_radii)

[8.35881546 8.29968126 8.38004795 ... 8.31472488 8.34391244 8.35219914]


In [ ]:
import math


def certify_sample(model, x, y_pred, sigma, num_samples, beta):
    """
    Computes the certified radius for a single input using randomized smoothing.

    Arguments:
        model: A PyTorch model to certify.
        x: The input tensor to certify.
        y_pred: The model's predicted output for the input x.
        sigma: The standard deviation of the Gaussian noise to add to the input during smoothing.
        num_samples: The number of samples to use for smoothing.
        beta: The confidence level for the certification, which determines the radius of the certified set.

    Returns:
        The certified radius for the input x.
    """

    # Set the model to evaluation mode
    model.eval()

    # Compute the perturbation standard deviation for the given sigma and beta
    delta = sigma * math.sqrt(num_samples - 1) * norm.ppf(1 - beta / 2)

    # Compute the smoothed predictions for the input x
    y_smooth = torch.zeros((num_samples,) + y_pred.shape, dtype=y_pred.dtype, device=y_pred.device)
    for i in range(num_samples):
        noise = torch.randn_like(x) * sigma
        y_smooth[i] = model(x + noise)
    y_smooth = y_smooth.softmax(dim=-1)

    # Compute the predicted class probabilities for the smoothed predictions
    y_bar = y_smooth.mean(dim=0)

    # Compute the distance between the original predictions and the smoothed predictions
    d = (y_pred.softmax(dim=-1) - y_bar).norm(p=2)

    # Compute the certified radius
    r = delta + d.item()

    return r


def certify(model, dataset, sigma, num_samples, beta):
    """
    Computes certified radii for a PyTorch model using randomized smoothing.

    Arguments:
        model: A PyTorch model to certify.
        dataset: A PyTorch dataset containing the data to certify.
        sigma: The standard deviation of the Gaussian noise to add to the input during smoothing.
        num_samples: The number of samples to use for smoothing.
        beta: The confidence level for the certification, which determines the radius of the certified set.

    Returns:
        A numpy array of certified radii, one for each input in the dataset.
    """

    # Set the model to evaluation mode
    model.eval()

    # Determine the size of the dataset
    n = len(dataset)

    # Compute the certified radii for each input in the dataset
    certified_radii = np.zeros(n)
    for i in range(n):
        x, y = dataset[i]
        x = x.unsqueeze(0).to(device)
        y_pred = model(x)
        r = certify_sample(model, x, y_pred, sigma, num_samples, beta)
        certified_radii[i] = r

    return certified_radii


# Generate the certified radii for the ResNet18 model on CIFAR-10

from torchvision.datasets import CIFAR10
from torchvision.transforms import ToTensor

# Load the CIFAR-10 test dataset
testset = CIFAR10(root='./data', train=False, download=True, transform=ToTensor())

# Create a data loader for the test dataset
testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=False)

# Define the standard deviation, number of samples, and confidence level for certification
sigma = 0.25
num_samples = 100
beta = 0.001

# Call the certify function with the test dataset and the certification parameters
certified_radii = certify(net, testset, sigma, num_samples, beta)



# Evaluate the ResNet18 model against PGD attacks
epsilons = [0.005, 0.01, 0.02, 0.04, 0.08, 0.16, 0.32]
for epsilon in epsilons:
    print(f"PGD Attack with epsilon={epsilon}")
    correct = 0
    total = 0
    net.eval()
    for batch_idx, (inputs, targets) in enumerate(testloader):
        inputs, targets = inputs.to(device), targets.to(device)
        adv_inputs = pgd_attack(net, inputs, targets, epsilon, 20, 0.01, device=1) # Use PGD to generate adversarial examples
        outputs = net(adv_inputs)
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        acc = 100. * correct / total
print(f"Accuracy under PGD attack with epsilon {epsilon} : {acc:.2f} %")


In [10]:
import torch
import numpy as np
from tqdm import tqdm
from scipy.stats import norm

import math
import torch
import torch.nn as nn

import torch.nn.functional as F

def pgd_attack(model, images, labels, eps, alpha, iters, device):
    """
    PGD attack function for adversarial examples generation

    Args:
    - model: the neural network model to attack
    - images: the clean images to be attacked
    - labels: the true labels of the images
    - eps: the maximum L-infinity norm of the perturbations
    - alpha: the step size of each iteration of the attack
    - iters: the number of iterations of the attack
    - device: the device to run the attack on (e.g., 'cuda' for GPU)

    Returns:
    - adv_images: the generated adversarial examples
    """
    # Set model to evaluation mode
    model.eval()

    # Generate random perturbations
    delta = torch.zeros_like(images).uniform_(-eps, eps)
    delta = torch.clamp(images + delta, min=0, max=1) - images
    delta.requires_grad = True

    for i in range(iters):
        # Forward pass
        outputs = model(images + delta)

        # Calculate loss
        loss = nn.CrossEntropyLoss()(outputs, labels)

        # Backward pass
        model.zero_grad()
        loss.backward()

        # Generate adversarial perturbation
        with torch.no_grad():
            delta += alpha * delta.grad.sign()
            delta = torch.clamp(delta, min=-eps, max=eps)
            delta = torch.clamp(images + delta, min=0, max=1) - images
            delta.requires_grad = True

    # Return the adversarial examples
    adv_images = torch.clamp(images + delta, min=0, max=1)

    return adv_images


epsilons = [0.005, 0.01, 0.02, 0.04, 0.08, 0.16, 0.32]
for epsilon in epsilons:
    print(f"PGD Attack with epsilon={epsilon}")
    correct = 0
    total = 0
    net.eval()
    for batch_idx, (inputs, targets) in enumerate(testloader):
        inputs, targets = inputs.to(device), targets.to(device)
        adv_inputs = pgd_attack(net, inputs, targets, epsilon, 0.01, 2, 1) # Use PGD to generate adversarial examples
        outputs = net(adv_inputs)
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
    acc = 100. * correct / total
    print(f"Accuracy under PGD attack with epsilon {epsilon} : {acc:.2f} %")


PGD Attack with epsilon=0.005
Accuracy under PGD attack with epsilon 0.005 : 13.47 %
PGD Attack with epsilon=0.01
Accuracy under PGD attack with epsilon 0.01 : 11.91 %
PGD Attack with epsilon=0.02
Accuracy under PGD attack with epsilon 0.02 : 10.39 %
PGD Attack with epsilon=0.04
Accuracy under PGD attack with epsilon 0.04 : 9.58 %
PGD Attack with epsilon=0.08
Accuracy under PGD attack with epsilon 0.08 : 9.17 %
PGD Attack with epsilon=0.16
Accuracy under PGD attack with epsilon 0.16 : 8.77 %
PGD Attack with epsilon=0.32
Accuracy under PGD attack with epsilon 0.32 : 8.00 %


In [12]:
import torch
import numpy as np
from tqdm import tqdm
from scipy.stats import norm

import math
import torch
import torch.nn as nn

import torch.nn.functional as F

def pgd_attack(model, images, labels, eps, alpha, iters, device):
    """
    PGD attack function for adversarial examples generation

    Args:
    - model: the neural network model to attack
    - images: the clean images to be attacked
    - labels: the true labels of the images
    - eps: the maximum L-infinity norm of the perturbations
    - alpha: the step size of each iteration of the attack
    - iters: the number of iterations of the attack
    - device: the device to run the attack on (e.g., 'cuda' for GPU)

    Returns:
    - adv_images: the generated adversarial examples
    """
    # Set model to evaluation mode
    model.eval()

    # Generate random perturbations
    delta = torch.zeros_like(images).uniform_(-eps, eps)
    delta = torch.clamp(images + delta, min=0, max=1) - images
    delta.requires_grad = True

    for i in range(iters):
        # Forward pass
        outputs = model(images + delta)

        # Calculate loss
        loss = nn.CrossEntropyLoss()(outputs, labels)

        # Backward pass
        model.zero_grad()
        loss.backward()

        # Generate adversarial perturbation
        with torch.no_grad():
            delta += alpha * delta.grad.sign()
            delta = torch.clamp(delta, min=-eps, max=eps)
            delta = torch.clamp(images + delta, min=0, max=1) - images
            delta.requires_grad = True

    # Return the adversarial examples
    adv_images = torch.clamp(images + delta, min=0, max=1)

    return adv_images


epsilons = [0.0005, 0.001, 0.002]
for epsilon in epsilons:
    print(f"PGD Attack with epsilon={epsilon}")
    correct = 0
    total = 0
    net.eval()
    for batch_idx, (inputs, targets) in enumerate(testloader):
        inputs, targets = inputs.to(device), targets.to(device)
        adv_inputs = pgd_attack(net, inputs, targets, epsilon, 0.01, 2, 1) # Use PGD to generate adversarial examples
        outputs = net(adv_inputs)
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
    acc = 100. * correct / total
    print(f"Accuracy under PGD attack with epsilon {epsilon} : {acc:.2f} %")


PGD Attack with epsilon=0.0005
Accuracy under PGD attack with epsilon 0.0005 : 14.99 %
PGD Attack with epsilon=0.001
Accuracy under PGD attack with epsilon 0.001 : 14.80 %
PGD Attack with epsilon=0.002
Accuracy under PGD attack with epsilon 0.002 : 14.44 %


In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
##from models import ResNet18 # Import your ResNet18 model
##from certify import certify # Import the certify function from the provided code
##from attacks import pgd_attack # Import the PGD attack function from the provided code

import torch
import torchvision.models as models

# Download ResNet18 and set the pretrained parameter to True
resnet18 = models.resnet18(pretrained=True)


# Set up the device (GPU or CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the CIFAR-10 dataset
transform = transforms.Compose([transforms.RandomCrop(32, padding=4),
                                transforms.RandomHorizontalFlip(),
                                transforms.ToTensor(),
                                transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])])
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=128, shuffle=False, num_workers=2)

# Define the ResNet18 model and optimizer
##net = ResNet18()
net = resnet18
net = net.to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)

# Train the ResNet18 model
for epoch in range(2):
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
    train_acc = 100. * correct / total

    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)
            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    test_acc = 100. * correct / total

    print('Epoch [%d/%d] Train Loss: %.3f Train Acc: %.3f Test Loss: %.3f Test Acc: %.3f' % (
        epoch + 1, 200, train_loss, train_acc, test_loss, test_acc))

###############################################################################
import torch
import numpy as np
from tqdm import tqdm
from scipy.stats import norm

'''def certify(model, x, sigma, num_samples, beta):
    """
    Compute the certified radius for a given input using randomized smoothing.

    Args:
        model (nn.Module): the neural network model to certify
        x (Tensor): the input image for which to compute the certified radius
        sigma (float): the standard deviation of the Gaussian noise used for randomized smoothing
        num_samples (int): the number of samples used to estimate the robustness of the smoothed classifier
        beta (float): the confidence level of the certification

    Returns:
        float: the certified radius of the input
    """
    # Set the model to evaluation mode
    model.eval()

    # Compute the standard deviation of the noise after normalization
    sigma_norm = sigma / np.sqrt(np.prod(x.shape[1:]))

    # Generate a batch of samples from the input with Gaussian noise
    x_samples = torch.tensor(
        np.random.normal(loc=x, scale=sigma_norm, size=(num_samples, *x.shape)).astype(np.float32)
    )

    # Evaluate the model on the noisy samples
    with torch.no_grad():
        logits = model(x_samples)

    # Compute the maximum logit across the samples
    max_logits, _ = logits.max(dim=1)

    # Compute the threshold for the certified radius
    threshold = norm.ppf(beta) * max_logits.std() + max_logits.mean()

    # Compute the certified radius
    radius = sigma * norm.ppf(beta) / (max_logits - threshold).abs().max().sqrt().item()

    return radius'''


sigma = 0.25
num_samples = 100
beta = 0.1

# Compute the certified radius for the first image in the test set
x, y = next(iter(testloader))
x, y = x.to(device), y.to(device)
radius = certify(resnet18, x[0], sigma, num_samples, beta)
print(f"Certified radius for image 0: {radius:.4f}")

###############################################################################

import torch
import torch.nn.functional as F

def pgd_attack(model, x, y, epsilon, alpha, num_iter):
    """
    PGD attack on a neural network model.

    Args:
        model (nn.Module): the neural network model to attack
        x (Tensor): the input image to attack
        y (Tensor): the true label of the input image
        epsilon (float): the maximum perturbation allowed
        alpha (float): the step size of the attack
        num_iter (int): the number of iterations of the attack

    Returns:
        Tensor: the perturbed image
    """
    # Create a copy of the input image for the attack
    x_adv = x.clone().detach()

    # Set the image to require gradient
    x_adv.requires_grad = True

    # PGD attack loop
    for i in range(num_iter):
        # Zero out the gradient
        model.zero_grad()

        # Forward pass to get the logits
        logits = model(x_adv)

        # Compute the loss as the cross-entropy between the logits and the true label
        loss = F.cross_entropy(logits, y)

        # Backward pass to compute the gradient
        loss.backward()

        # Compute the sign of the gradient
        grad = x_adv.grad.detach().sign()

        # Update the perturbed image
        x_adv = x_adv + alpha * grad

        # Clip the perturbation to epsilon
        x_adv = torch.max(torch.min(x_adv, x + epsilon), x - epsilon)

        # Clamp the perturbed image to [0, 1]
        x_adv = torch.clamp(x_adv, 0, 1)

        # Detach the gradient to prevent it from accumulating
        x_adv = x_adv.detach()

    return x_adv

###############################################################################



import math


def certify_sample(model, x, y_pred, sigma, num_samples, beta):
    """
    Computes the certified radius for a single input using randomized smoothing.

    Arguments:
        model: A PyTorch model to certify.
        x: The input tensor to certify.
        y_pred: The model's predicted output for the input x.
        sigma: The standard deviation of the Gaussian noise to add to the input during smoothing.
        num_samples: The number of samples to use for smoothing.
        beta: The confidence level for the certification, which determines the radius of the certified set.

    Returns:
        The certified radius for the input x.
    """

    # Set the model to evaluation mode
    model.eval()

    # Compute the perturbation standard deviation for the given sigma and beta
    delta = sigma * math.sqrt(num_samples - 1) * norm.ppf(1 - beta / 2)

    # Compute the smoothed predictions for the input x
    y_smooth = torch.zeros((num_samples,) + y_pred.shape, dtype=y_pred.dtype, device=y_pred.device)
    for i in range(num_samples):
        noise = torch.randn_like(x) * sigma
        y_smooth[i] = model(x + noise)
    y_smooth = y_smooth.softmax(dim=-1)

    # Compute the predicted class probabilities for the smoothed predictions
    y_bar = y_smooth.mean(dim=0)

    # Compute the distance between the original predictions and the smoothed predictions
    d = (y_pred.softmax(dim=-1) - y_bar).norm(p=2)

    # Compute the certified radius
    r = delta + d.item()

    return r


def certify(model, dataset, sigma, num_samples, beta):
    """
    Computes certified radii for a PyTorch model using randomized smoothing.

    Arguments:
        model: A PyTorch model to certify.
        dataset: A PyTorch dataset containing the data to certify.
        sigma: The standard deviation of the Gaussian noise to add to the input during smoothing.
        num_samples: The number of samples to use for smoothing.
        beta: The confidence level for the certification, which determines the radius of the certified set.

    Returns:
        A numpy array of certified radii, one for each input in the dataset.
    """

    # Set the model to evaluation mode
    model.eval()

    # Determine the size of the dataset
    n = len(dataset)

    # Compute the certified radii for each input in the dataset
    certified_radii = np.zeros(n)
    for i in range(n):
        x, y = dataset[i]
        x = x.unsqueeze(0).to(device)
        y_pred = model(x)
        r = certify_sample(model, x, y_pred, sigma, num_samples, beta)
        certified_radii[i] = r

    return certified_radii


# Generate the certified radii for the ResNet18 model on CIFAR-10

from torchvision.datasets import CIFAR10
from torchvision.transforms import ToTensor

# Load the CIFAR-10 test dataset
testset = CIFAR10(root='./data', train=False, download=True, transform=ToTensor())

# Create a data loader for the test dataset
testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=False)

# Define the standard deviation, number of samples, and confidence level for certification
sigma = 0.25
num_samples = 100
beta = 0.001

# Call the certify function with the test dataset and the certification parameters
certified_radii = certify(net, testset, sigma, num_samples, beta)

# Evaluate the ResNet18 model against PGD attacks
epsilons = [0.005, 0.01, 0.02, 0.04, 0.08, 0.16, 0.32]
for epsilon in epsilons:
    print(f"PGD Attack with epsilon={epsilon}")
    correct = 0
    total = 0
    net.eval()
    for batch_idx, (inputs, targets) in enumerate(testloader):
        inputs, targets = inputs.to(device), targets.to(device)
        adv_inputs = pgd_attack(net, inputs, targets, epsilon, 20, 0.01, device=1) # Use PGD to generate adversarial examples
        outputs = net(adv_inputs)
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        acc = 100. * correct / total
print(f"Accuracy under PGD attack with epsilon {epsilon} : {acc:.2f} %")


/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 60.2MB/s]


100%|██████████| 170498071/170498071 [00:03<00:00, 49929596.58it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
Epoch [1/200] Train Loss: 1012.324 Train Acc: 20.586 Test Loss: 154.123 Test Acc: 30.360
Epoch [2/200] Train Loss: 675.178 Train Acc: 34.890 Test Loss: 129.226 Test Acc: 39.350


NameError: ignored

In [ ]:
!curl -O https://s3.amazonaws.com/fast-ai-imageclas/imagenette2.tgz
!tar xzf imagenette2.tgz


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1485M  100 1485M    0     0  41.3M      0  0:00:35  0:00:35 --:--:-- 42.9M


In [ ]:
import torchvision.datasets as datasets

imagenette = datasets.ImageFolder('imagenette2/train')


In [ ]:

'''
This code will evaluate the smoothed classifier on the test set and print out the test accuracy. 
Note that this may take some time to run, depending on the size of the test set and the number of samples used for smoothing.
'''


import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import numpy as np

# Define the Gaussian data augmentation function
def gaussian_noise(image, sigma):
    return image + torch.randn_like(image) * sigma

# Define the dataset class for ImageNet
class ImageNetDataset(Dataset):
    def __init__(self, root, transform=None):
        self.transform = transform
        self.dataset = torchvision.datasets.ImageFolder(root=root, transform=transform)
        
    def __getitem__(self, index):
        image, label = self.dataset[index]
        image = gaussian_noise(image, sigma)
        return image, label
    
    def __len__(self):
        return len(self.dataset)

'''import torchvision.datasets as datasets

imagenette = datasets.ImageFolder('imagenette2/train')'''

# Load the ImageNet dataset
transform = transforms.Compose([transforms.Resize(256), transforms.CenterCrop(224), transforms.ToTensor()])
train_dataset = ImageNetDataset(root='imagenette2/train', transform=transform)
test_dataset = ImageNetDataset(root='imagenette2/val', transform=transform)

# Define the neural network f and train it
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = torch.nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = torch.nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.fc1 = torch.nn.Linear(64*56*56, 512)
        self.fc2 = torch.nn.Linear(512, 1000)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = x.view(-1, 64*56*56)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Net().to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

sigma = 0.1
n_samples = 100
##smooth_classifier = SmoothClassifier(model, sigma)


##num_epochs = 10
num_epochs = 3
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}: Loss = {running_loss/len(train_loader)}")

# Define the smoothed classifier g
class SmoothClassifier(torch.nn.Module):
    def __init__(self, f, sigma):
        super(SmoothClassifier, self).__init__()
        self.f = f
        self.sigma = sigma
        
    def forward(self, x):
        logits = torch.zeros((len(x), 1000)).to(device)
        for i in range(n_samples):
            noise = self.sigma * torch.randn_like(x).to(device)
            logits += self.f(x + noise)
        return logits / n_samples

'''sigma = 0.1
n_samples = 100
'''


# Test the smoothed classifier on an example image
import matplotlib.pyplot as plt

def test_smoothed_classifier(image, model, n_samples, sigma):
    # Smooth the image using the model
    smoothed_image = randomized_smoothing(image, model, n_samples, sigma)

    # Get the predicted label from the smoothed image
    smoothed_label = torch.argmax(smoothed_image).item()

    # Get the confidence of the prediction
    smoothed_confidence = smoothed_image.max().item()

    # Plot the original and smoothed images side by side
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
    ax1.imshow(image.cpu().squeeze().permute(1, 2, 0))
    ax1.set_title('Original Image')
    ax2.imshow(smoothed_image.cpu().squeeze().permute(1, 2, 0))
    ax2.set_title(f'Smoothed Image (Label: {smoothed_label}, Confidence: {smoothed_confidence:.2f})')
    plt.show()


'''# Get a batch of test images
images, labels = next(iter(test_loader))

# Pass the first image in the batch to the smoothed classifier test function
test_smoothed_classifier(images[0], model, n_samples, sigma)

# Load an example image from the test set
x, _ = next(iter(test_loader))

# Test the smoothed classifier on the example image
test_smoothed_classifier(x, model, n_samples, sigma)


# Test the smoothed classifier on the example image
test_smoothed_classifier(x, model, n_samples, sigma)'''

# Evaluate the smoothed classifier on the test set
import torchvision.datasets as datasets
import torchvision.transforms as transforms

# Define the test set
test_dataset = datasets.ImageNet(root='imagenet', split='val', transform=transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
]))

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

# Evaluate the smoothed classifier on the test set
def evaluate_smoothed_classifier(model, test_loader, n_samples, sigma):
    num_correct = 0
    num_total = 0

    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        # Smooth the images using the model
        smoothed_images = randomized_smoothing(images, model, n_samples, sigma)

        # Get the predicted labels from the smoothed images
        smoothed_labels = torch.argmax(smoothed_images, dim=1)

        # Compute the accuracy of the smoothed classifier
        num_correct += (smoothed_labels == labels).sum().item()
        num_total += labels.size(0)

    accuracy = num_correct / num_total
    return accuracy

test_accuracy = evaluate_smoothed_classifier(model, test_loader, n_samples, sigma)
print(f'Test Accuracy: {test_accuracy:.2f}')


Epoch 1: Loss = 2.506537323062484
Epoch 2: Loss = 1.6559423812337823
Epoch 3: Loss = 1.3265508791884861
Epoch 4: Loss = 0.9965532989920797
Epoch 5: Loss = 0.6760593493645256
Epoch 6: Loss = 0.3954530240514794
Epoch 7: Loss = 0.21401000249426108
Epoch 8: Loss = 0.11071349250001682
Epoch 9: Loss = 0.06186388568902338
Epoch 10: Loss = 0.04959902284994117


NameError: ignored

In [ ]:

# Define the smoothed classifier g
class SmoothClassifier(torch.nn.Module):
    def __init__(self, f, sigma):
        super(SmoothClassifier, self).__init__()
        self.f = f
        self.sigma = sigma
        
    def forward(self, x):
        logits = torch.zeros((len(x), 1000)).to(device)
        for i in range(n_samples):
            noise = self.sigma * torch.randn_like(x).to(device)
            logits += self.f(x + noise)
        return logits / n_samples

'''sigma = 0.1
n_samples = 100
'''


# Test the smoothed classifier on an example image
import matplotlib.pyplot as plt

def test_smoothed_classifier(image, model, n_samples, sigma):
    # Smooth the image using the model
    smoothed_image = randomized_smoothing(image, model, n_samples, sigma)

    # Get the predicted label from the smoothed image
    smoothed_label = torch.argmax(smoothed_image).item()

    # Get the confidence of the prediction
    smoothed_confidence = smoothed_image.max().item()

    # Plot the original and smoothed images side by side
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
    ax1.imshow(image.cpu().squeeze().permute(1, 2, 0))
    ax1.set_title('Original Image')
    ax2.imshow(smoothed_image.cpu().squeeze().permute(1, 2, 0))
    ax2.set_title(f'Smoothed Image (Label: {smoothed_label}, Confidence: {smoothed_confidence:.2f})')
    plt.show()


'''# Get a batch of test images
images, labels = next(iter(test_loader))

# Pass the first image in the batch to the smoothed classifier test function
test_smoothed_classifier(images[0], model, n_samples, sigma)

# Load an example image from the test set
x, _ = next(iter(test_loader))

# Test the smoothed classifier on the example image
test_smoothed_classifier(x, model, n_samples, sigma)


# Test the smoothed classifier on the example image
test_smoothed_classifier(x, model, n_samples, sigma)'''

# Evaluate the smoothed classifier on the test set
import torchvision.datasets as datasets
import torchvision.transforms as transforms

# Define the test set
test_dataset = datasets.ImageNet(root='imagenette2', split='val', transform=transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
]))

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

# Evaluate the smoothed classifier on the test set
def evaluate_smoothed_classifier(model, test_loader, n_samples, sigma):
    num_correct = 0
    num_total = 0

    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        # Smooth the images using the model
        smoothed_images = randomized_smoothing(images, model, n_samples, sigma)

        # Get the predicted labels from the smoothed images
        smoothed_labels = torch.argmax(smoothed_images, dim=1)

        # Compute the accuracy of the smoothed classifier
        num_correct += (smoothed_labels == labels).sum().item()
        num_total += labels.size(0)

    accuracy = num_correct / num_total
    return accuracy

test_accuracy = evaluate_smoothed_classifier(model, test_loader, n_samples, sigma)
print(f'Test Accuracy: {test_accuracy:.2f}')


RuntimeError: ignored

In [ ]:
# 1.7 
# Repeat the autoencoder steps with a nonlinear autoencoder and compare the results against PCA and linear encoder.

!pip install torch numpy matplotlib sklearn
from torchvision.datasets import FashionMNIST
from torchvision import transforms
import torch
import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
transform = transforms.Compose([transforms.ToTensor()])

trainset = FashionMNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

testset = FashionMNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)
class NonlinearAutoencoder(torch.nn.Module):
    def __init__(self):
        super(NonlinearAutoencoder, self).__init__()
        self.encoder = torch.nn.Sequential(
            torch.nn.Linear(28 * 28, 256),
            torch.nn.ReLU(),
            torch.nn.Linear(256, 128),
            torch.nn.ReLU(),
            torch.nn.Linear(128, 10)
        )
        self.decoder = torch.nn.Sequential(
            torch.nn.Linear(10, 128),
            torch.nn.ReLU(),
            torch.nn.Linear(128, 256),
            torch.nn.ReLU(),
            torch.nn.Linear(256, 28 * 28),
            torch.nn.Sigmoid()
        )
    def forward(self, x):
        x = x.view(-1, 28 * 28)
        x = self.encoder(x)
        x = self.decoder(x)
        x = x.view(-1, 1, 28, 28)
        return x

nonlinear_autoencoder = NonlinearAutoencoder()
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(nonlinear_autoencoder.parameters(), lr=0.001)

for epoch in range(10):
    for data in trainloader:
        img, _ = data
        optimizer.zero_grad()
        output = nonlinear_autoencoder(img)
        loss = criterion(output, img)
        loss.backward()
        optimizer.step()
    print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, 10, loss.item()))

hidden_representations = []
labels = []

for data in testloader:
    img, label = data
    output = autoencoder.encoder(img.view(-1, 28 * 28))
    hidden_representations.append(output.detach().numpy())
    labels.append(label.numpy())

hidden_representations = np.concatenate(hidden_representations, axis=0)
labels = np.concatenate(labels, axis=0)

tsne = TSNE(n_components=2)
hidden_tsne = tsne.fit_transform(hidden_representations)

plt.figure(figsize=(10, 10))
for i in range(10):
    plt.scatter(hidden_tsne[labels == i, 0], hidden_tsne[labels == i, 1], label=str(i))
plt.legend()
plt.show()


In [ ]:
!git clone https://github.com/fastai/imagenette
!pip install torchvision
import tarfile

with tarfile.open('/content/imagenette/imagenette2-160.tgz', 'r:gz') as tar:
    tar.extractall('/content/named_folders/imagenet-100')


In [ ]:
!wget http://www.image-net.org/challenges/LSVRC/2012/nnoupb/ILSVRC2012_img_train.tar
!tar -xvf ILSVRC2012_img_train.tar

!wget http://www.image-net.org/challenges/LSVRC/2012/nnoupb/ILSVRC2012_img_val.tar
!tar -xvf ILSVRC2012_img_val.tar

import torchvision.datasets as datasets
import os

!mkdir imagenet
# Define the path to the ImageNet data
data_path = './imagenet/'

# Define the subset of classes to use
classes = ['cat', 'dog', 'bird']

# Create a PyTorch dataset using the ImageFolder class
dataset = datasets.ImageFolder(root=data_path, 
                                transform=transforms.Compose([
                                    transforms.Resize(256),
                                    transforms.CenterCrop(224),
                                    transforms.ToTensor(),
                                    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                                                         std=[0.229, 0.224, 0.225])
                                ]), 
                                target_transform=lambda x: classes.index(os.path.basename(os.path.dirname(x))))

# Split the dataset into training and validation sets
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [len(dataset)-500, 500])


In [ ]:
import torchvision.datasets as datasets
import torchvision.transforms as transforms

# Define the ImageNet dataset
train_dataset = datasets.ImageNet(root='imagenet', split='train', transform=transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
]))

# Download the dataset
train_dataset.download()


In [ ]:

'''
This code will evaluate the smoothed classifier on the test set and print out the test accuracy. 
Note that this may take some time to run, depending on the size of the test set and the number of samples used for smoothing.
'''


import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import numpy as np

# Define the Gaussian data augmentation function
def gaussian_noise(image, sigma):
    return image + torch.randn_like(image) * sigma

# Define the dataset class for ImageNet
class ImageNetDataset(Dataset):
    def __init__(self, root, transform=None):
        self.transform = transform
        self.dataset = torchvision.datasets.ImageFolder(root=root, transform=transform)
        
    def __getitem__(self, index):
        image, label = self.dataset[index]
        image = gaussian_noise(image, sigma)
        return image, label
    
    def __len__(self):
        return len(self.dataset)

# Load the ImageNet dataset
transform = transforms.Compose([transforms.Resize(256), transforms.CenterCrop(224), transforms.ToTensor()])
train_dataset = ImageNetDataset(root='path/to/imagenet/train', transform=transform)
test_dataset = ImageNetDataset(root='path/to/imagenet/val', transform=transform)

# Define the neural network f and train it
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = torch.nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = torch.nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.fc1 = torch.nn.Linear(64*56*56, 512)
        self.fc2 = torch.nn.Linear(512, 1000)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = x.view(-1, 64*56*56)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Net().to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

num_epochs = 10
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}: Loss = {running_loss/len(train_loader)}")

# Define the smoothed classifier g
class SmoothClassifier(torch.nn.Module):
    def __init__(self, f, sigma):
        super(SmoothClassifier, self).__init__()
        self.f = f
        self.sigma = sigma
        
    def forward(self, x):
        logits = torch.zeros((len(x), 1000)).to(device)
        for i in range(n_samples):
            noise = self.sigma * torch.randn_like(x).to(device)
            logits += self.f(x + noise)
        return logits / n_samples

sigma = 0.1
n_samples = 100



# Test the smoothed classifier on an example image
import matplotlib.pyplot as plt

def test_smoothed_classifier(image, model, n_samples, sigma):
    # Smooth the image using the model
    smoothed_image = randomized_smoothing(image, model, n_samples, sigma)

    # Get the predicted label from the smoothed image
    smoothed_label = torch.argmax(smoothed_image).item()

    # Get the confidence of the prediction
    smoothed_confidence = smoothed_image.max().item()

    # Plot the original and smoothed images side by side
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
    ax1.imshow(image.cpu().squeeze().permute(1, 2, 0))
    ax1.set_title('Original Image')
    ax2.imshow(smoothed_image.cpu().squeeze().permute(1, 2, 0))
    ax2.set_title(f'Smoothed Image (Label: {smoothed_label}, Confidence: {smoothed_confidence:.2f})')
    plt.show()

# Test the smoothed classifier on the example image
test_smoothed_classifier(x, model, n_samples, sigma)

# Evaluate the smoothed classifier on the test set
import torchvision.datasets as datasets
import torchvision.transforms as transforms

# Define the test set
test_dataset = datasets.ImageNet(root='imagenet', split='val', transform=transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
]))

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

# Evaluate the smoothed classifier on the test set
def evaluate_smoothed_classifier(model, test_loader, n_samples, sigma):
    num_correct = 0
    num_total = 0

    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        # Smooth the images using the model
        smoothed_images = randomized_smoothing(images, model, n_samples, sigma)

        # Get the predicted labels from the smoothed images
        smoothed_labels = torch.argmax(smoothed_images, dim=1)

        # Compute the accuracy of the smoothed classifier
        num_correct += (smoothed_labels == labels).sum().item()
        num_total += labels.size(0)

    accuracy = num_correct / num_total
    return accuracy

test_accuracy = evaluate_smoothed_classifier(model, test_loader, n_samples, sigma)
print(f'Test Accuracy: {test_accuracy:.2f}')


In [ ]:
!pip install torch torchvision

In [ ]:


#!pip install torch torchvision

import torch
import torchvision.models as models

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = models.resnet50(pretrained=True).to(device)
model.eval()


import numpy as np
import torch.nn.functional as F

def randomized_smoothing(x, model, n_samples, noise_std):
    logits = torch.zeros((n_samples,) + model(x).shape).to(device)
    for i in range(n_samples):
        noise = noise_std * torch.randn(x.shape).to(device)
        logits[i] = model(x + noise)
    avg_logits = torch.mean(logits, dim=0)
    return F.softmax(avg_logits, dim=-1)


from PIL import Image
import requests
from io import BytesIO

url = 'https://upload.wikimedia.org/wikipedia/commons/3/32/Tom_Cruise_by_Gage_Skidmore.jpg'
response = requests.get(url)
img = Image.open(BytesIO(response.content))
img = img.resize((224, 224), resample=Image.BILINEAR)

x = torch.tensor(np.array(img)).permute(2, 0, 1).unsqueeze(0).float().div(255).to(device)


n_samples = 100
noise_std = 0.1

y_smooth = randomized_smoothing(x, model, n_samples, noise_std)
pred_label = torch.argmax(y_smooth).item()

print(f"Predicted class: {pred_label}")


In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
import numpy as np
import torch.nn.functional as F

# Define the standard deviation of the Gaussian noise
sigma = 0.1

# Define the number of samples for randomized smoothing
n_samples = 50

# Define the batch size for evaluation
batch_size = 64

# Define the device to run the model on
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the ImageNet dataset with data augmentation
transform_train = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
trainset = torchvision.datasets.ImageFolder(root='./imagenet/train', transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)

# Train a ResNet50 model on the dataset
model = models.resnet50(pretrained=False).to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=1e-4)
for epoch in range(10):
    for i, (inputs, labels) in enumerate(trainloader):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

# Define the randomized smoothing function
def smoothed_classifier(x, model, n_samples, sigma):
    logits = torch.zeros((n_samples,) + model(x).shape).to(device)
    for i in range(n_samples):
        noise = sigma * torch.randn(x.shape).to(device)
        logits[i] = model(x + noise)
    avg_logits = torch.mean(logits, dim=0)
    return F.softmax(avg_logits, dim=-1)

# Evaluate the smoothed classifier on the test set
transform_test = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
testset = torchvision.datasets.ImageFolder(root='./imagenet/val', transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

total, correct = 0, 0
for i, (inputs, labels) in enumerate(testloader):
    inputs, labels = inputs.to(device), labels.to(device)
    outputs = smoothed_classifier(inputs, model, n_samples, sigma)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()
accuracy = 100 * correct / total
print('Accuracy of the smoothed classifier on the test set: %.2f%%' % accuracy)
